In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/train-label-aihub/TL_0220.zip
/kaggle/input/train-label-aihub/TL_0177.zip
/kaggle/input/train-label-aihub/TL_0231.zip
/kaggle/input/train-label-aihub/TL_0778.zip
/kaggle/input/train-label-aihub/TL_0638.zip
/kaggle/input/train-label-aihub/TL_0216.zip
/kaggle/input/train-label-aihub/TL_0026.zip
/kaggle/input/train-label-aihub/TL_0104.zip
/kaggle/input/train-label-aihub/TL_0349.zip
/kaggle/input/train-label-aihub/TL_0064.zip
/kaggle/input/train-label-aihub/TL_0284.zip
/kaggle/input/train-label-aihub/TL_0129.zip
/kaggle/input/train-label-aihub/TL_0365.zip
/kaggle/input/train-label-aihub/TL_0348.zip
/kaggle/input/train-label-aihub/TL_0494.zip
/kaggle/input/train-label-aihub/TL_0174.zip
/kaggle/input/train-label-aihub/TL_0286.zip
/kaggle/input/train-label-aihub/TL_0301.zip
/kaggle/input/train-label-aihub/TL_0516.zip
/kaggle/input/train-label-aihub/TL_0036.zip
/kaggle/input/train-label-aihub/TL_0314.zip
/kaggle/input/train-label-aihub/TL_0133.zip
/kaggle/input/train-label-aihub/

> # Upload this ipynb to GitHub under your GitHub account won-hj. This can only be undone directly on GitHub.

MIT License
Copyright (c) <year> <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

참고?
https://www.kaggle.com/code/diaaessam/face-verification-and-recognition 
https://www.kaggle.com/code/atulanandjha/face-recognition-starter-trick-code-with-output
https://www.kaggle.com/datasets/atulanandjha/lfwpeople


#import 
# 이미지 로드 및 전처리하기
- 먼저 고급 Keras 사전 처리 유틸리티(예: tf.keras.utils.image_dataset_from_directory) 및 레이어(예: tf.keras.layers.Rescaling)를 사용하여 디스크에서 이미지 디렉터리를 읽습니다.
- 다음으로 tf.data를 사용하여 처음부터 고유한 입력 파이프라인을 작성합니다.
- 마지막으로 TensorFlow Datasets에서 사용할 수 있는 대규모 카탈로그에서 데이터세트를 다운로드합니다.

In [2]:
import keras
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

2024-05-19 08:16:42.453539: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 08:16:42.453803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 08:16:42.653176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import zipfile
import os
import glob

filename = {'train-label':'TL_',
            'train-original':'TS_', 
            'validation-labelling':'VL_', 
            'validation-original':'VS_'}

for k,v in filename.items():
    file_path = '/kaggle/input/{}-aihub/'.format(k)   #.join(glob.glob('.zip'))#.join('-aihub/TL_').join(glob.glob('.zip'))
    print(file_path)
    dir_path = '/kaggle/working/{}/'.format(k)
    print(dir_path)
    
    
    # 지정된 디렉토리가 없으면 생성합니다.
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    #print(os.getcwd())
    # zip 파일 압축 해제
    # 각 압축파일마다 압축해제 
    files = os.listdir(file_path)
    print(files)
    
    for i in range(len(files)):
        with zipfile.ZipFile(file_path+files[i], 'r') as zip_ref:
            zip_ref.extractall(dir_path)

/kaggle/input/train-label-aihub/
/kaggle/working/train-label/
['TL_0220.zip', 'TL_0177.zip', 'TL_0231.zip', 'TL_0778.zip', 'TL_0638.zip', 'TL_0216.zip', 'TL_0026.zip', 'TL_0104.zip', 'TL_0349.zip', 'TL_0064.zip', 'TL_0284.zip', 'TL_0129.zip', 'TL_0365.zip', 'TL_0348.zip', 'TL_0494.zip', 'TL_0174.zip', 'TL_0286.zip', 'TL_0301.zip', 'TL_0516.zip', 'TL_0036.zip', 'TL_0314.zip', 'TL_0133.zip', 'TL_0179.zip', 'TL_0412.zip', 'TL_0395.zip', 'TL_0137.zip', 'TL_0842.zip', 'TL_0325.zip', 'TL_0217.zip', 'TL_0583.zip', 'TL_0099.zip', 'TL_0039.zip', 'TL_0311.zip', 'TL_0245.zip', 'TL_0189.zip', 'TL_0242.zip', 'TL_0853.zip', 'TL_0241.zip', 'TL_0624.zip', 'TL_0865.zip', 'TL_0103.zip', 'TL_0872.zip', 'TL_0033.zip', 'TL_0628.zip', 'TL_0536.zip', 'TL_0222.zip', 'TL_0116.zip', 'TL_0221.zip', 'TL_0539.zip', 'TL_0282.zip', 'TL_0096.zip', 'TL_0780.zip', 'TL_0310.zip', 'TL_0637.zip', 'TL_0075.zip', 'TL_0681.zip', 'TL_0626.zip', 'TL_0047.zip', 'TL_0775.zip', 'TL_0297.zip', 'TL_0518.zip', 'TL_0016.zip', 'TL_024

In [4]:
#데이터셋의 데이터 구조를 출력해보고 모델에 돌리기 

#print(os.dirlist('kaggle/working/train-original'))
#!pwd
#!rm *.json
#!ls 
#!ls train-labelling
#!rm -rf train-lebelling
#!ls validation-labelling
!cat /kaggle/working/train-label/0016_1968_55_00000060_D.json

'''
0016_1968_55_00000060_D.json -> id_출생년도_나이_
** 크기 조정 후 바운딩 박스나 랜드마크가 변경되는지 확인 **
{
    "filename": "0016_1968_55_00000060_D",
    "id": 16,
    "birth": 1968,
    "age_now": 55,
    "age_past": 55,
    "format": "png",
    "imgsize": "1.324MB",
    "width": 1080,
    "height": 1440,
    "device": "digital",
    "gender": "male",
    "annotation": [
        {
            "box": {
                "x": 3.047921471137073,
                "y": 469.34504908053935,
                "w": 474.59417858244757,
                "h": 519.9905464947254
            },
            "landmark": [
                [
                    311.88763427734375,
                    722.7200927734375
                ],
                [
                    169.94879515695465,
                    559.9612768464077
                ],
                [
                    397.5303955078125,
                    587.8244458053117
                ],
                [
                    165.16746397340108,
                    823.2084790084367
                ],
                [
                    355.03024047511775,
                    823.2270239167203
                ]
            ]
        }
    ]
}

'''

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()














































}

'\n0016_1968_55_00000060_D.json -> id_출생년도_나이_\n** 크기 조정 후 바운딩 박스나 랜드마크가 변경되는지 확인 **\n{\n    "filename": "0016_1968_55_00000060_D",\n    "id": 16,\n    "birth": 1968,\n    "age_now": 55,\n    "age_past": 55,\n    "format": "png",\n    "imgsize": "1.324MB",\n    "width": 1080,\n    "height": 1440,\n    "device": "digital",\n    "gender": "male",\n    "annotation": [\n        {\n            "box": {\n                "x": 3.047921471137073,\n                "y": 469.34504908053935,\n                "w": 474.59417858244757,\n                "h": 519.9905464947254\n            },\n            "landmark": [\n                [\n                    311.88763427734375,\n                    722.7200927734375\n                ],\n                [\n                    169.94879515695465,\n                    559.9612768464077\n                ],\n                [\n                    397.5303955078125,\n                    587.8244458053117\n                ],\n                [\n               